# Helper functions for training alphas

In [ ]:
import CSV
import DataFrames: DataFrame
import JLD2
import Memoize: @memoize
import Setfield
import Setfield: @set
import StatsBase
import SparseArrays

## Static data

In [ ]:
get_data_path(file) = joinpath(@__DIR__, "../../data/$file");

In [ ]:
@memoize function num_users()
    df = DataFrame(CSV.File(get_data_path("processed_data/username_to_uid.csv")))
    length(df.uid)
end

@memoize function num_items(medium)
    df = DataFrame(CSV.File(get_data_path("processed_data/$(medium)_to_uid.csv")))
    length(df.uid)
end;

In [ ]:
@memoize function get_timestamp_encodings()
    function parse_line(file::IO, field::String, format::Type = Int)
        line = readline(file)
        fields = split(strip(line), ",")
        @assert length(fields) == 2
        @assert fields[1] == field
        parse(format, fields[2])
    end

    open(get_data_path("processed_data/timestamps.csv")) do f
        return parse_line(f, "min_timestamp"), parse_line(f, "max_timestamp")
    end
end

@memoize function day_in_timestamp_units()
    min_timestamp, max_timestamp = get_timestamp_encodings()
    seconds_in_day = 24 * 60 * 60
    return seconds_in_day / Float64(max_timestamp - min_timestamp)
end

days_in_timestamp_units(d) = d * day_in_timestamp_units()

function timestamp_to_unix(ts)
    @assert ts > 0
    min_timestamp, max_timestamp = get_timestamp_encodings()
    unix_time = Int64(round(ts * (max_timestamp - min_timestamp) + min_timestamp))
end

function unix_to_timestamp(unix_time)
    min_timestamp, max_timestamp = get_timestamp_encodings()
    ts = (unix_time - min_timestamp) / (max_timestamp - min_timestamp)
    convert(Float32, ts)
end

function timestamp_to_date(ts)
    Dates.unix2datetime(timestamp_to_unix(ts))
end;

In [ ]:
@memoize function get_status(status::Symbol)
    status_encoding = Dict(
        :rewatching => 7,
        :completed => 6,
        :watching => 5,
        :on_hold => 4,
        :plan_to_watch => 3,
        :dropped => 2,
        :wont_watch => 1,
        :none => 0,
    )
    status_encoding[status]
end;

## Reading and writing data

In [ ]:
@kwdef struct RatingsDataset
    source::Vector{Int32} = []
    medium::Vector{Int32} = []
    userid::Vector{Int32} = []
    itemid::Vector{Int32} = []
    status::Vector{Int32} = []
    rating::Vector{Float32} = []
    update_order::Vector{Int32} = []
    updated_at::Vector{Float32} = []
    created_at::Vector{Float32} = []
    started_at::Vector{Float32} = []
    finished_at::Vector{Float32} = []
    progress::Vector{Float32} = []
    repeat_count::Vector{Int32} = []
    priority::Vector{Float32} = []
    sentiment::Vector{Int32} = []
    sentiment_score::Vector{Float32} = []
    owned::Vector{Float32} = []
    metric::Vector{Float32} = []
    alpha::Vector{Float32} = []
end;

# append two datasets
function Base.cat(x::RatingsDataset, y::RatingsDataset)
    # argument validation
    x_is_nonempty = false
    y_is_nonempty = false
    for field in fieldnames(RatingsDataset)
        if length(getfield(x, field)) != 0
            x_is_nonempty = true
        end
        if length(getfield(y, field)) != 0
            y_is_nonempty = true
        end
    end
    if x_is_nonempty && y_is_nonempty
        for field in fieldnames(RatingsDataset)
            a = length(getfield(x, field)) != 0
            b = length(getfield(y, field)) != 0
            @assert a == b "cat: missing field $field"
        end
    end

    RatingsDataset(
        [vcat(getfield(x, c), getfield(y, c)) for c in fieldnames(RatingsDataset)]...,
    )
end

function Base.filter(x::RatingsDataset, mask::BitVector)
    filter_array(a) = length(a) > 0 ? a[mask] : []
    RatingsDataset([filter_array(getfield(x, c)) for c in fieldnames(RatingsDataset)]...)
end;

In [ ]:
function training_test_split(df::RatingsDataset)
    c = get_settings()
    if c["mode"] == "research"
        ts_cutoff = days_in_timestamp_units(c["cutoff_days"] * 2)
    else
        ts_cutoff = days_in_timestamp_units(c["cutoff_days"])
    end
    test_mask =
        (df.update_order .<= c["cutoff_interactions"]) .&& (df.updated_at .>= 1 - ts_cutoff)
    filter(df, .!(test_mask)), filter(df, test_mask)
end;

In [ ]:
function SparseArrays.sparse(x::RatingsDataset, medium)
    SparseArrays.sparse(
        x.itemid .+ 1,
        x.userid .+ 1,
        x.metric,
        num_items(medium),
        num_users(),
    )
end;

In [ ]:
const ALL_SPLITS = ["training", "test", "negative"]
const ALL_METRICS = ["rating", "watch", "plantowatch", "drop"]
const ALL_MEDIUMS = ["manga", "anime"];

In [ ]:
function get_raw_split(
    split::String,
    medium::String,
    fields::Vector{Symbol},
    alpha::Union{String,Nothing},
)
    df = RatingsDataset()
    basepath = get_data_path(split)
    for i = 1:length(fields)
        df = Setfield.set(
            df,
            Setfield.PropertyLens{fields[i]}(),
            JLD2.load(
                "$basepath/splits/$split.$medium.$(fields[i]).jld2",
                String(fields[i]),
            ),
        )
    end
    if !isnothing(alpha)
        alpha_df = JLD2.load(
            "$basepath/alphas/$(alpha)/alpha.jld2",
            "$split.$medium";
            # need a typemap because of daemon mode
            typemap = Dict("Main.anonymous.RatingsDataset" => RatingsDataset),
        )
        if length(fields) > 0
            @assert length(getfield(df, fields[1])) == length(alpha_df.alpha)
        end
        df = @set df.alpha = alpha_df.alpha
    end
    df
end

function get_raw_split(split::Dict, medium::String, fields::Vector{Symbol}, alpha::Nothing)
    # todo update in compress_media_lists
    field_map = Dict("itemid" => "mediaid", "update_order" => "backward_order")
    # convert from an http response
    d = split[medium]
    df = RatingsDataset()
    basepath = get_data_path(split)
    for i = 1:length(fields)
        key = get(field_map, String(fields[i]), String(fields[i]))
        df = Setfield.set(df, Setfield.PropertyLens{fields[i]}(), d[key])
    end
    df
end;

In [ ]:
function get_split(
    split::Union{String,Dict},
    metric::String,
    medium::String,
    fields::Vector{Symbol},
    alpha::Union{String,Nothing} = nothing,
)
    validargs =
        (split in ALL_SPLITS || isa(split, Dict)) &&
        metric in ALL_METRICS &&
        medium in ALL_MEDIUMS
    @assert validargs "($split, $metric, $medium) ∉ (split, metric, medium)"

    extrafields = Dict(
        "rating" => [:rating],
        "watch" => [:status],
        "plantowatch" => [:status],
        "drop" => [:status],
    )
    fetch = union(Set(fields), Set(extrafields[metric]))
    delete!(fetch, :metric)
    df = get_raw_split(split, medium, collect(fetch), alpha)

    if metric == "rating"
        df = filter(df, df.rating .!= 0)
        df = @set df.metric = copy(df.rating)
    elseif metric == "watch"
        df = filter(df, df.status .> get_status(:plan_to_watch))
        df = @set df.metric = ones(Float32, length(df.status))
    elseif metric == "plantowatch"
        df = filter(df, df.status .== get_status(:plan_to_watch))
        df = @set df.metric = ones(Float32, length(df.status))
    elseif metric == "drop"
        df = @set df.metric = zeros(Float32, length(df.status))
        df = filter(df, df.status .> get_status(:none))
        for i = 1:length(df.status)
            if df.status[i] <= get_status(:dropped)
                df.metric[i] = 1
            end
        end
    else
        @assert false
    end

    # clear any columns you are not interested in
    if !isnothing(alpha)
        push!(fields, :alpha)
    end
    for f in fieldnames(RatingsDataset)
        if f ∉ fields
            df = Setfield.set(df, Setfield.PropertyLens{f}(), [])
        end
    end
    df
end;

In [ ]:
function write_alpha(
    model::Function,
    medium::String,
    outdir::String,
    splits::Vector{String},
    username::String = "",
    source::String = "",
)
    alpha = Dict()
    for split in splits
        df = get_raw_split(split, medium, [:userid, :itemid], nothing, username, source)
        x = model(df.userid, df.itemid)
        @assert length(x) == length(df.userid)
        alpha["$split.$medium"] = RatingsDataset(alpha = x)
    end
    basepath = get_data_path(splits[1], username, source)
    outdir = mkpath("$basepath/alphas/$outdir")
    JLD2.save("$outdir/alpha.jld2", alpha; compress = true)
end;

In [ ]:
function read_alpha(alpha::String, split::String, metric::String, medium::String)
    get_split(split, metric, medium, [:userid, :itemid], alpha)
end;

In [ ]:
function write_params(params, outdir, environment)
    if environment
        outdir = mkpath(get_data_path("../environment/alphas/$outdir"))
    else
        outdir = mkpath(get_data_path("alphas/$outdir"))
    end
    JLD2.save("$outdir/params.jld2", params; compress = true)
end

function read_params(alpha, environment)
    if environment
        fn = get_data_path("../environment/alphas/$alpha/params.jld2")
    else
        fn = get_data_path("alphas/$alpha/params.jld2")
    end
    JLD2.load(fn)
end;

In [ ]:
function powerdecay(x, a)
    x == 0 ? zero(eltype(a)) : sign(x) * abs(x)^a
end

function powerdecay(x::Vector, a)
    y = Array{eltype(a)}(undef, length(x))
    for i = 1:length(x)
        y[i] = powerdecay(x[i], a)
    end
    y
end

function get_counts(data::Vector{Int32})
    counts = StatsBase.countmap(data)
    ret = zeros(Int32, length(data))
    for i = 1:length(data)
        ret[i] = counts[data[i]]
    end
    ret
end;